# 模拟退火算法解决TSP问题：北京大学校园路径规划

该Notebook实现了基于模拟退火算法的旅行商问题(TSP)解决方案，包括：

1. **模拟退火算法**：通过模拟物理退火过程，在概率性接受更差解的机制下跳出局部最优
2. **多种邻域操作**：2-opt交换、插入操作、逆序操作等生成新解
3. **动态温度控制**：指数冷却策略，平衡探索与利用
4. **可视化展示**：展示退火过程和最终路径结果
5. **性能对比**：与随机路径和贪心算法进行对比分析

## 1. 环境设置与数据配置

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import time
import math
from collections import deque
import copy

# Matplotlib 中文显示设置
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 地图文件路径
MAP_IMAGE_PATH = 'map_1.0.jpg'

# 北京大学地点坐标
CITIES = {
    "West Gate": (703, 595),
    "Weiming Lake": (1069, 630), 
    "Jing Yuan": (928, 830),
    "Boya Tower": (1148, 646),
    "Statue of the Former Principal Cai Yuanpei": (885, 632),
    "Zhibeizi Garden": (1320, 1052),
    "YANNANYUAN": (969, 991),
    "May 4th Playground": (1251, 1147),
    "University Library": (1083, 816),
    "Stone Fish": (959, 632),
    "Tomb of Mr. Edgar Snow": (1070, 730),
    "Marble Boat": (1040, 571),
    "Luce Pavilion": (1012, 582),
    "Tian Yuan": (1314, 670),
    "MINGHE YUAN": (733, 518),
    "Campus Scenery Pavilion": (840, 470),
    "LANGRUN YUAN": (1134, 271),
    "JINGCHUN YUAN": (945, 455),
    "Lotus Pond": (750, 432),
    "SHAO HAI": (736, 742),
    "Catering Building": (939, 1111),
    "Humanities Garden": (1155, 433),
    "Statue of Professor Li Dazhao": (926, 738),
    "Museum of University History": (785,713),
    "University Hall": (1098, 992),
    "North Shore": (1049, 519),
    "The Bridge of Magpies": (993, 559),
    "Silent Wall": (1118, 503),
    "Fountain": (878, 1062),
    "Red Lake": (864, 419),
    "The Institute of Poetry Studies": (993, 358),
    "Tan Siu Lin Center for International Studies": (1080, 697)
}

print(f"北京大学地点总数: {len(CITIES)}")
print("地点列表:")
for i, (name, coord) in enumerate(CITIES.items(), 1):
    print(f"{i:2d}. {name:<40} {coord}")

## 2. 模拟退火算法实现

In [ ]:
class SimulatedAnnealingTSP:
    def __init__(self, cities):
        self.cities = cities
        self.city_names = list(cities.keys())
        self.city_coords = np.array(list(cities.values()))
        self.num_cities = len(cities)
        
        # 计算距离矩阵
        self.distance_matrix = self._calculate_distance_matrix()
        
        # 退火过程记录
        self.temperature_history = []
        self.cost_history = []
        self.best_cost_history = []
        self.acceptance_history = []
        
    def _calculate_distance_matrix(self):
        """计算城市间的欧几里得距离矩阵"""
        distances = np.zeros((self.num_cities, self.num_cities))
        for i in range(self.num_cities):
            for j in range(self.num_cities):
                if i != j:
                    coord1 = self.city_coords[i]
                    coord2 = self.city_coords[j]
                    distances[i][j] = np.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)
        return distances
    
    def calculate_path_distance(self, path):
        """计算给定路径的总距离"""
        total_distance = 0
        for i in range(len(path)):
            current_city = path[i]
            next_city = path[(i + 1) % len(path)]  # 回到起点
            total_distance += self.distance_matrix[current_city][next_city]
        return total_distance
    
    def generate_initial_solution(self, start_city=0):
        """生成初始解（随机排列）"""
        cities = list(range(self.num_cities))
        cities.remove(start_city)
        random.shuffle(cities)
        return [start_city] + cities
    
    def two_opt_swap(self, path, i, j):
        """2-opt邻域操作：逆转路径中两点间的顺序"""
        new_path = path.copy()
        new_path[i:j+1] = reversed(new_path[i:j+1])
        return new_path
    
    def insertion_move(self, path):
        """插入操作：将一个城市插入到另一个位置"""
        new_path = path.copy()
        # 随机选择一个非起点城市进行移动
        move_indices = list(range(1, len(path)))  # 不移动起点
        from_idx = random.choice(move_indices)
        to_idx = random.choice([i for i in range(1, len(path)) if i != from_idx])
        
        city = new_path.pop(from_idx)
        new_path.insert(to_idx, city)
        return new_path
    
    def swap_cities(self, path):
        """交换操作：交换两个非起点城市的位置"""
        new_path = path.copy()
        # 只交换非起点城市
        swap_indices = list(range(1, len(path)))
        if len(swap_indices) >= 2:
            idx1, idx2 = random.sample(swap_indices, 2)
            new_path[idx1], new_path[idx2] = new_path[idx2], new_path[idx1]
        return new_path
    
    def generate_neighbor(self, path):
        """生成邻近解"""
        operation = random.choice(['2-opt', 'insertion', 'swap'])
        
        if operation == '2-opt' and len(path) > 3:
            # 确保不包含起点在逆转范围内
            i = random.randint(1, len(path) - 2)
            j = random.randint(i + 1, len(path) - 1)
            return self.two_opt_swap(path, i, j)
        elif operation == 'insertion':
            return self.insertion_move(path)
        else:  # swap
            return self.swap_cities(path)
    
    def acceptance_probability(self, current_cost, new_cost, temperature):
        """计算接受概率（Metropolis准则）"""
        if new_cost < current_cost:
            return 1.0
        else:
            if temperature <= 0:
                return 0.0
            return math.exp(-(new_cost - current_cost) / temperature)
    
    def simulated_annealing(self, start_city=0, initial_temp=10000, final_temp=1, 
                           cooling_rate=0.995, max_iterations_per_temp=100, 
                           max_iterations=50000, verbose=True):
        """模拟退火主算法"""
        print(f"\n=== 模拟退火算法 ===")
        print(f"起始城市: {self.city_names[start_city]}")
        print(f"初始温度: {initial_temp}")
        print(f"最终温度: {final_temp}")
        print(f"冷却系数: {cooling_rate}")
        print(f"每温度最大迭代数: {max_iterations_per_temp}")
        print(f"总最大迭代数: {max_iterations}")
        
        start_time = time.time()
        
        # 初始化
        current_solution = self.generate_initial_solution(start_city)
        current_cost = self.calculate_path_distance(current_solution)
        
        best_solution = current_solution.copy()
        best_cost = current_cost
        
        temperature = initial_temp
        iteration = 0
        temperature_changes = 0
        total_accepted = 0
        total_generated = 0
        
        # 清空历史记录
        self.temperature_history = []
        self.cost_history = []
        self.best_cost_history = []
        self.acceptance_history = []
        
        print(f"\n初始解距离: {current_cost:.2f}")
        print("\n开始退火过程...")
        
        # 主循环
        while temperature > final_temp and iteration < max_iterations:
            accepted_at_temp = 0
            iterations_at_temp = 0
            
            # 在当前温度下进行多次迭代
            for _ in range(max_iterations_per_temp):
                if iteration >= max_iterations:
                    break
                    
                # 生成新解
                new_solution = self.generate_neighbor(current_solution)
                new_cost = self.calculate_path_distance(new_solution)
                
                # 计算接受概率
                accept_prob = self.acceptance_probability(current_cost, new_cost, temperature)
                
                # 决定是否接受新解
                if random.random() < accept_prob:
                    current_solution = new_solution
                    current_cost = new_cost
                    accepted_at_temp += 1
                    total_accepted += 1
                    
                    # 更新最优解
                    if current_cost < best_cost:
                        best_solution = current_solution.copy()
                        best_cost = current_cost
                        if verbose and iteration % 1000 == 0:
                            print(f"迭代 {iteration:5d}: 新最优解 = {best_cost:.2f}, 温度 = {temperature:.2f}")
                
                total_generated += 1
                iterations_at_temp += 1
                iteration += 1
                
                # 记录历史
                if iteration % 100 == 0:  # 每100次迭代记录一次
                    self.temperature_history.append(temperature)
                    self.cost_history.append(current_cost)
                    self.best_cost_history.append(best_cost)
                    self.acceptance_history.append(total_accepted / total_generated if total_generated > 0 else 0)
            
            # 降温
            temperature *= cooling_rate
            temperature_changes += 1
            
            # 显示温度变化信息
            if verbose and temperature_changes % 50 == 0:
                acceptance_rate = accepted_at_temp / iterations_at_temp if iterations_at_temp > 0 else 0
                print(f"温度 {temperature:.2f}: 接受率 = {acceptance_rate:.3f}, 当前解 = {current_cost:.2f}, 最优解 = {best_cost:.2f}")
        
        execution_time = time.time() - start_time
        final_acceptance_rate = total_accepted / total_generated if total_generated > 0 else 0
        
        print(f"\n退火完成!")
        print(f"算法执行时间: {execution_time:.4f} 秒")
        print(f"总迭代次数: {iteration}")
        print(f"温度变化次数: {temperature_changes}")
        print(f"最终温度: {temperature:.6f}")
        print(f"总体接受率: {final_acceptance_rate:.4f}")
        print(f"最优路径距离: {best_cost:.2f}")
        
        return best_solution, best_cost
    
    def random_path(self, start_city=0):
        """生成随机路径作为对比"""
        cities = list(range(self.num_cities))
        cities.remove(start_city)
        random.shuffle(cities)
        path = [start_city] + cities
        total_distance = self.calculate_path_distance(path)
        return path, total_distance

# 创建TSP求解器实例
sa_tsp_solver = SimulatedAnnealingTSP(CITIES)
print("模拟退火TSP求解器初始化完成！")
print(f"距离矩阵大小: {sa_tsp_solver.distance_matrix.shape}")

## 3. 运行模拟退火算法

In [ ]:
# 选择起始点 (West Gate)
start_city_name = "West Gate"
start_city_index = sa_tsp_solver.city_names.index(start_city_name)

print(f"选择起始地点: {start_city_name} (索引: {start_city_index})")
print("="*100)

# 设置随机种子以便结果可复现
random.seed(42)
np.random.seed(42)

# 运行模拟退火算法
sa_path, sa_distance = sa_tsp_solver.simulated_annealing(
    start_city=start_city_index,
    initial_temp=15000,      # 初始温度
    final_temp=0.1,          # 最终温度  
    cooling_rate=0.995,      # 冷却系数
    max_iterations_per_temp=50,  # 每温度最大迭代数
    max_iterations=30000,    # 总最大迭代数
    verbose=True
)

print("\n" + "="*100)

# 生成随机路径作为对比
print("\n=== 随机路径对比 ===")
random_paths = []
random_distances = []

for i in range(5):
    rand_path, rand_distance = sa_tsp_solver.random_path(start_city_index)
    random_paths.append(rand_path)
    random_distances.append(rand_distance)
    print(f"随机路径 {i+1}: 距离 = {rand_distance:.2f}")

avg_random_distance = np.mean(random_distances)
best_random_distance = min(random_distances)

print(f"\n随机路径统计:")
print(f"  平均距离: {avg_random_distance:.2f}")
print(f"  最佳距离: {best_random_distance:.2f}")

print("\n" + "="*100)
print("🏆 算法性能对比")
print("="*100)
print(f"模拟退火算法:       {sa_distance:.2f}")
print(f"随机路径平均:       {avg_random_distance:.2f}")
print(f"随机路径最佳:       {best_random_distance:.2f}")

print(f"\n📊 性能提升:")
sa_improvement = (avg_random_distance - sa_distance) / avg_random_distance * 100
print(f"模拟退火 vs 随机平均: {sa_improvement:.1f}% 改善")

sa_vs_best_random = (best_random_distance - sa_distance) / best_random_distance * 100
if sa_distance < best_random_distance:
    print(f"模拟退火 vs 随机最佳: {sa_vs_best_random:.1f}% 改善")
else:
    print(f"模拟退火未能超越随机最佳解")

## 4. 退火过程可视化

In [ ]:
# 绘制退火过程图
if sa_tsp_solver.temperature_history and sa_tsp_solver.cost_history:
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. 温度变化
    iterations = range(0, len(sa_tsp_solver.temperature_history) * 100, 100)
    ax1.plot(iterations, sa_tsp_solver.temperature_history, 'r-', linewidth=2)
    ax1.set_title('温度变化过程', fontsize=14, fontweight='bold')
    ax1.set_xlabel('迭代次数')
    ax1.set_ylabel('温度')
    ax1.grid(True, alpha=0.3)
    ax1.set_yscale('log')  # 使用对数刻度
    
    # 2. 成本变化
    ax2.plot(iterations, sa_tsp_solver.cost_history, 'b-', linewidth=2, alpha=0.7, label='当前解')
    ax2.plot(iterations, sa_tsp_solver.best_cost_history, 'g-', linewidth=3, label='最优解')
    ax2.set_title('解的质量变化', fontsize=14, fontweight='bold')
    ax2.set_xlabel('迭代次数')
    ax2.set_ylabel('路径距离')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. 接受率变化
    ax3.plot(iterations, sa_tsp_solver.acceptance_history, 'purple', linewidth=2)
    ax3.set_title('接受率变化', fontsize=14, fontweight='bold')
    ax3.set_xlabel('迭代次数')
    ax3.set_ylabel('接受率')
    ax3.grid(True, alpha=0.3)
    ax3.set_ylim(0, 1)
    
    # 4. 温度vs接受率散点图
    ax4.scatter(sa_tsp_solver.temperature_history, sa_tsp_solver.acceptance_history, 
               alpha=0.6, c=range(len(sa_tsp_solver.temperature_history)), cmap='viridis')
    ax4.set_title('温度与接受率关系', fontsize=14, fontweight='bold')
    ax4.set_xlabel('温度')
    ax4.set_ylabel('接受率')
    ax4.set_xscale('log')
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("警告: 没有收集到退火过程数据")

## 5. 结果可视化

In [ ]:
def plot_tsp_result(tsp_solver, path, distance, title, algorithm_name):
    """绘制TSP结果"""
    plt.figure(figsize=(16, 12))
    
    # 绘制地图背景
    try:
        campus_map = mpimg.imread(MAP_IMAGE_PATH)
        plt.imshow(campus_map, extent=[0, campus_map.shape[1], campus_map.shape[0], 0])
    except FileNotFoundError:
        print(f"警告: 地图文件 '{MAP_IMAGE_PATH}' 未找到")
        plt.xlim(0, 1500)
        plt.ylim(0, 1200)
        plt.gca().invert_yaxis()
    
    # 绘制路径
    path_coords = tsp_solver.city_coords[path + [path[0]]]  # 添加回到起点
    plt.plot(path_coords[:, 0], path_coords[:, 1], 'r-', linewidth=3, alpha=0.8, label=f'{algorithm_name}路径')
    
    # 绘制城市点
    plt.scatter(tsp_solver.city_coords[:, 0], tsp_solver.city_coords[:, 1], 
               c='blue', s=100, zorder=5, alpha=0.8, edgecolors='white', linewidth=2)
    
    # 突出显示起点
    start_coord = tsp_solver.city_coords[path[0]]
    plt.scatter(start_coord[0], start_coord[1], c='green', s=200, marker='s', 
               zorder=6, label='起点', edgecolors='white', linewidth=2)
    
    # 添加城市编号（访问顺序）
    for i, city_idx in enumerate(path):
        coord = tsp_solver.city_coords[city_idx]
        plt.annotate(f'{i+1}', (coord[0], coord[1]), xytext=(5, 5), 
                    textcoords='offset points', fontsize=10, fontweight='bold',
                    bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7))
    
    plt.title(f'{title}\n总距离: {distance:.2f}', fontsize=16, fontweight='bold')
    plt.xlabel('X坐标', fontsize=14)
    plt.ylabel('Y坐标', fontsize=14)
    plt.legend(fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    return plt.gcf()

# 绘制模拟退火算法结果
fig_sa = plot_tsp_result(sa_tsp_solver, sa_path, sa_distance, 
                        '模拟退火算法 - 北京大学校园路径规划', '模拟退火')

## 6. 详细路径分析

In [ ]:
def print_detailed_path_analysis(tsp_solver, path, distance, algorithm_name):
    """打印详细的路径分析"""
    print(f"\n{'='*80}")
    print(f"🔍 {algorithm_name} - 详细路径分析")
    print(f"{'='*80}")
    
    print(f"\n📍 完整访问路径:")
    total_segments = len(path)
    cumulative_distance = 0
    
    for i in range(total_segments):
        current_city = path[i]
        next_city = path[(i + 1) % total_segments]  # 回到起点
        
        current_name = tsp_solver.city_names[current_city]
        next_name = tsp_solver.city_names[next_city]
        
        segment_distance = tsp_solver.distance_matrix[current_city][next_city]
        cumulative_distance += segment_distance
        
        status = "[回到起点]" if i == total_segments - 1 else ""
        print(f"{i+1:2d}. {current_name:<40} -> {next_name:<40} "
              f"距离: {segment_distance:7.2f} | 累计: {cumulative_distance:8.2f} {status}")
    
    print(f"\n📊 统计信息:")
    segments_distances = [tsp_solver.distance_matrix[path[i]][path[(i+1) % len(path)]] 
                         for i in range(len(path))]
    
    print(f"   总访问地点数: {len(path)}")
    print(f"   总路径距离: {distance:.2f}")
    print(f"   平均段距离: {np.mean(segments_distances):.2f}")
    print(f"   最长段距离: {np.max(segments_distances):.2f}")
    print(f"   最短段距离: {np.min(segments_distances):.2f}")
    print(f"   距离标准差: {np.std(segments_distances):.2f}")
    
    # 找出最长和最短的路径段
    max_segment_idx = np.argmax(segments_distances)
    min_segment_idx = np.argmin(segments_distances)
    
    print(f"\n🔗 路径段分析:")
    print(f"   最长路径段: {tsp_solver.city_names[path[max_segment_idx]]} -> "
          f"{tsp_solver.city_names[path[(max_segment_idx+1) % len(path)]]} ({segments_distances[max_segment_idx]:.2f})")
    print(f"   最短路径段: {tsp_solver.city_names[path[min_segment_idx]]} -> "
          f"{tsp_solver.city_names[path[(min_segment_idx+1) % len(path)]]} ({segments_distances[min_segment_idx]:.2f})")

# 分析模拟退火算法结果
print_detailed_path_analysis(sa_tsp_solver, sa_path, sa_distance, "模拟退火算法")

## 7. 算法性能分析与参数敏感性测试

In [ ]:
# 测试不同参数组合的性能
print("\n" + "="*100)
print("🧪 参数敏感性分析")
print("="*100)

# 定义测试参数组合
parameter_combinations = [
    {"name": "高温慢冷", "initial_temp": 20000, "cooling_rate": 0.999, "max_iter": 20000},
    {"name": "中温中冷", "initial_temp": 10000, "cooling_rate": 0.995, "max_iter": 20000},
    {"name": "低温快冷", "initial_temp": 5000, "cooling_rate": 0.990, "max_iter": 20000},
    {"name": "超高温", "initial_temp": 30000, "cooling_rate": 0.995, "max_iter": 25000}
]

results_comparison = []

for i, params in enumerate(parameter_combinations):
    print(f"\n🔬 测试 {i+1}: {params['name']}")
    print(f"   初始温度: {params['initial_temp']}, 冷却系数: {params['cooling_rate']}, 最大迭代: {params['max_iter']}")
    
    # 运行多次取平均
    distances = []
    times = []
    
    for run in range(3):  # 运行3次
        start_time = time.time()
        path, distance = sa_tsp_solver.simulated_annealing(
            start_city=start_city_index,
            initial_temp=params['initial_temp'],
            final_temp=0.1,
            cooling_rate=params['cooling_rate'],
            max_iterations_per_temp=30,
            max_iterations=params['max_iter'],
            verbose=False
        )
        exec_time = time.time() - start_time
        
        distances.append(distance)
        times.append(exec_time)
        print(f"     运行 {run+1}: {distance:.2f} ({exec_time:.2f}s)")
    
    avg_distance = np.mean(distances)
    std_distance = np.std(distances)
    avg_time = np.mean(times)
    
    results_comparison.append({
        'name': params['name'],
        'avg_distance': avg_distance,
        'std_distance': std_distance,
        'avg_time': avg_time,
        'best_distance': min(distances)
    })
    
    print(f"   平均距离: {avg_distance:.2f} ± {std_distance:.2f}")
    print(f"   最佳距离: {min(distances):.2f}")
    print(f"   平均时间: {avg_time:.2f}s")

# 参数性能对比
print(f"\n{'='*80}")
print("📊 参数组合性能排名")
print(f"{'='*80}")

# 按平均距离排序
sorted_results = sorted(results_comparison, key=lambda x: x['avg_distance'])

for i, result in enumerate(sorted_results, 1):
    print(f"{i}. {result['name']:<12}: 平均 {result['avg_distance']:7.2f} | "
          f"最佳 {result['best_distance']:7.2f} | 时间 {result['avg_time']:5.2f}s")

best_config = sorted_results[0]
print(f"\n🏆 最佳参数配置: {best_config['name']}")
print(f"   建议用于生产环境的参数组合")

## 8. 与其他算法对比分析

In [ ]:
# 创建算法性能对比图
plt.figure(figsize=(14, 10))

# 准备数据（这里我们使用之前计算的结果，实际中可以运行其他算法进行对比）
algorithms = ['模拟退火', '随机路径(平均)', '随机路径(最佳)']
distances = [sa_distance, avg_random_distance, best_random_distance]
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']

# 如果有其他算法结果，可以添加到这里
# 例如: algorithms.append('贪心算法'), distances.append(greedy_distance)

bars = plt.bar(algorithms, distances, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)

# 在柱状图上添加数值标签
for bar, distance in zip(bars, distances):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 50,
             f'{distance:.1f}', ha='center', va='bottom', fontweight='bold', fontsize=12)

plt.title('模拟退火算法与其他方法的TSP求解性能对比', fontsize=16, fontweight='bold', pad=20)
plt.ylabel('总路径距离', fontsize=14)
plt.xlabel('算法类型', fontsize=14)
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

# 打印算法特性对比
print("\n" + "="*100)
print("🎯 模拟退火算法 vs 其他算法特性对比")
print("="*100)

comparison_table = [
    ["算法特性", "模拟退火", "贪心算法", "随机搜索", "蚁群算法"],
    ["时间复杂度", "O(n×iter)", "O(n²)", "O(1)", "O(n²×iter)"],
    ["空间复杂度", "O(n)", "O(n²)", "O(n)", "O(n²)"],
    ["全局最优能力", "高", "低", "很低", "高"], 
    ["参数敏感性", "中等", "低", "无", "高"],
    ["收敛速度", "中等", "快", "不收敛", "慢"],
    ["实现复杂度", "中等", "简单", "简单", "复杂"],
    ["适用场景", "中大规模", "小规模", "基准测试", "复杂约束"]
]

for row in comparison_table:
    print(f"{row[0]:<12} | {row[1]:<12} | {row[2]:<12} | {row[3]:<12} | {row[4]:<12}")
    if row[0] == "算法特性":
        print("-" * 80)

## 9. 算法总结与应用建议

In [ ]:
# 打印最终总结
print("\n" + "="*100)
print("🎯 模拟退火算法TSP求解 - 最终总结")
print("="*100)

print(f"\n🏛️ 问题规模:")
print(f"   北京大学校园地点数: {len(CITIES)}")
print(f"   起始地点: {start_city_name}")
print(f"   问题复杂度: O(n!) = O({len(CITIES)}!) ≈ {np.math.factorial(len(CITIES)):.2e}")

print(f"\n🚀 算法性能:")
print(f"   模拟退火算法: {sa_distance:.2f}")
print(f"   随机解平均性能: {avg_random_distance:.2f}")
print(f"   性能提升: {sa_improvement:.1f}%")

print(f"\n🔥 模拟退火算法特点:")
print(f"   ✓ 全局搜索能力: 通过概率性接受差解跳出局部最优")
print(f"   ✓ 参数可调性: 温度、冷却策略、邻域操作可灵活配置")
print(f"   ✓ 收敛保证: 理论上能以概率1收敛到全局最优")
print(f"   ✓ 通用性强: 适用于各种组合优化问题")
print(f"   ✗ 收敛较慢: 需要大量迭代才能得到高质量解")
print(f"   ✗ 参数敏感: 性能高度依赖参数选择")

print(f"\n🎛️ 关键参数影响:")
print(f"   📈 初始温度: 影响初期探索能力，过高浪费时间，过低易陷入局部最优")
print(f"   ❄️ 冷却速度: 影响收敛速度和解质量，快速冷却收敛快但质量差")
print(f"   🔄 迭代次数: 影响算法充分性，次数越多解越好但时间越长")
print(f"   🎯 邻域操作: 影响搜索效率，多样化操作有助于跳出局部最优")

print(f"\n💡 算法改进建议:")
print(f"   🔧 自适应冷却: 根据接受率动态调整冷却速度")
print(f"   🎲 多样化邻域: 结合多种邻域操作(2-opt, 3-opt, insertion等)")
print(f"   🔄 重启机制: 在算法陷入停滞时重新开始")
print(f"   🧠 混合策略: 结合其他启发式算法(如遗传算法)")

print(f"\n🎨 可视化输出:")
print(f"   生成了详细的路径规划图")
print(f"   展示了退火过程的动态变化")
print(f"   包含访问顺序、距离信息和算法收敛过程")

print(f"\n🌟 应用场景:")
print(f"   🗺️ 路径规划: 物流配送、旅游路线、巡检路径")
print(f"   🏭 生产调度: 作业车间调度、任务分配")
print(f"   📐 工程设计: 电路设计、网络拓扑优化")
print(f"   🧬 生物信息: 序列比对、蛋白质折叠")

print("\n" + "="*100)
print("✅ 模拟退火算法TSP求解程序执行完成！")
print("💭 算法成功展示了通过模拟物理退火过程解决复杂优化问题的能力")
print("🔬 为进一步研究和应用提供了完整的实现框架和性能分析")
print("="*100)